In [10]:
import os
import re
import pandas as pd
import numpy as np
import tensorflow as tf
# from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TFGPT2Model, GPT2Config

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', lang='fr')

In [13]:
current_dir = os.path.dirname(os.path.realpath("__file__"))
repo_dir = os.path.dirname(current_dir)
DATA_FILE = os.path.join(repo_dir,"data","cleaned_data.csv")
data = pd.read_csv(DATA_FILE, encoding="utf-8",sep=',')

In [17]:
data['Type'] = df["Type"].to_list()

In [14]:
data

,Nom,Duree,Personnes,Type,Ingredients,Ustensiles,Etapes,recettes
0,soupe veloute potimarron pomme terre,45 min,4,4,2 pomme terre 2 oignon hache 3 gousse ail ha...,louche mixeur plonger blender chauffer couteau...,etape 1 enlever ecorce pepin potimarron obliga...,soupe veloute potimarron pomme terre 2 pomme ...
1,soupe oignon,45 min,4,4,4 oignon 50 gramme beurre 1 c.a. huile 1...,louche mixeur plonger blender chauffer saladie...,etape 1 pelez emincer oignon etape 2 faire rev...,soupe oignon 4 oignon 50 gramme beurre 1 ...
2,veloute potiron,1 h 15 min,4,4,1 oignon 1 quartier potiron 1 brique creme f...,louche mixeur plonger couteau cocotte fonte po...,etape 1 couper chair potiron gros de etape 2 c...,veloute potiron 1 oignon 1 quartier potiron...
3,veloute butternut cannelle,35 min,4,4,1 oignon 1 c.a. creme frais 1 pincee canne...,louche mixeur plonger couteau cuillere bois po...,etape 1 couper butternut gros cube etape 2 cou...,veloute butternut cannelle 1 oignon 1 c.a. ...
4,tarte poireau lardon,50 min,6,4,250 gramme farine 140 gramme beurre 2 c....,moule tarte four couteau pinceau rape poele sa...,etape 1 prechauffer four 210 degre c thermosta...,tarte poireau lardon 250 gramme farine 14...
...,...,...,...,...,...,...,...,...
47335,liqueur genepi,10 min,1,2,40 morceau sucre 40 brin genepi 1 l alco...,entonnoir balance cuisine,etape 1 transvaser litre alcool gros bouteille...,liqueur genepi 40 morceau sucre 40 brin g...
47336,caipirinha,3 min,1,2,1 dose jus citron vert frais 15 millilitre j...,cuillere bois shaker,etape 1 shaker melanger ingredient glace pilee...,caipirinha 1 dose jus citron vert frais 15 ...
47337,kombucha maison citron gingembre,-,1,2,1 morceau gingembre raper 100 gramme sucre...,casserole balance cuisine,etape 1 commencer realiser sirop chauffer eau ...,kombucha maison citron gingembre 1 morceau ...
47338,gin tonic,5 min,1,2,4 cl gin 2 rondelle citron 8 cl tonic,NaN,etape 1 verser glaçon verre etape 2 ajouter gi...,gin tonic 4 cl gin 2 rondelle citron 8 ...


In [16]:
data['tokens'] = data['recettes'].apply(tokenizer.tokenize)

In [15]:
oov_tokens = ['s','Entrée']
tokenizer.add_tokens(oov_tokens)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

max_length = 512
sequences = tokenizer.batch_encode_plus(data['recettes'].to_list(), padding=True, truncation=True, return_tensors='tf')

# num_classes = len(np.unique(y))
# y = tf.keras.utils.to_categorical(y, num_classes)

In [41]:
sequences['input_ids'].shape

TensorShape([47340, 1024])

In [16]:
# Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(sequences['input_ids'].numpy(), sequences['attention_mask'].numpy(), test_size=0.2, random_state=42)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

X = sequences['input_ids'].numpy()
y = sequences['attention_mask'].numpy()
input_dim = X.shape[1]

# Reshape the input data to a 3D tensor
batch_size = X.shape[0]
sequence_length = 1  # Set the sequence length to 1
# X = np.reshape(X, (batch_size, sequence_length, input_dim))

output_dim = y.shape[1]

# Reshape the target data to a 2D tensor
# Y = np.reshape(y, (batch_size, output_dim))

# Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
X_train

array([[[   83, 10602,   502, ..., 50257, 50257, 50257]],

       [[37424,    68,  1093, ..., 50257, 50257, 50257]],

       [[   70,   559, 19503, ..., 50257, 50257, 50257]],

       ...,

       [[   79,   378, 17463, ..., 50257, 50257, 50257]],

       [[  462, 21348,  3450, ..., 50257, 50257, 50257]],

       [[   66, 14501, 48241, ..., 50257, 50257, 50257]]])

In [17]:
model = TFGPT2Model.from_pretrained('gpt2')
config = GPT2Config.from_pretrained('gpt2')
config.output_hidden_states = True
model.config = config

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [19]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(
  loss=loss,
  optimizer=optimizer,
  metrics=metric
)

# Train the model
history = model.fit(
  sequences['input_ids'],
  sequences['attention_mask'],
  # batch_size=batch_size,
  epochs=10,
  # validation_data=(X_test, y_test)
)


Epoch 1/10


AttributeError: in user code:

    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\transformers\modeling_tf_utils.py", line 1476, in train_step
        data = data_adapter.expand_1d(data)

    AttributeError: module 'keras.engine.data_adapter' has no attribute 'expand_1d'


In [20]:
from keras.preprocessing.text import Tokenizer

# Initialize the Tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the instructions and named entities
tokenizer.fit_on_texts(data['recettes'])

# Convert the instructions and named entities to sequences of integers
X_recettes = tokenizer.texts_to_sequences(data['recettes'])
# X_entities = tokenizer.texts_to_sequences(df['entities'])


In [24]:
from keras.utils import pad_sequences

# Determine the maximum length of the sequences
max_length = max(len(seq) for seq in X_recettes)

# Pad or truncate the sequences to the maximum length
X_recettes = pad_sequences(X_recettes, maxlen=max_length)

In [25]:
from keras.layers import Embedding, LSTM, Dense
from keras import Sequential

# Initialize the model
model = Sequential()

# Add an embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_length))

# Add an LSTM layer
model.add(LSTM(units=128, return_sequences=True))

# Add a dense layer
model.add(Dense(units=len(data['recettes'].unique()), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [27]:
# Fit the model on the training data
model.fit(X_recettes, data['Type'], epochs=10, batch_size=32)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert' defined at (most recent call last):
    File "C:\Users\badje\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\badje\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP\projet\Recipe_generator\venv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
    File "c:\Users\badje\OneDrive\Bureau\Cours_M2\NLP